# Parallel Computing

## General thoughts

Parallel computing is a programming method that **harnesses the power of multiple processors (typically CPU cores) at once**.

There are many types of parallelism, some of which are (from micro to macro)

* **Instruction level parallelism** (e.g. SIMD)
* **Multi-threading** (shared memory)
* **Multi-processing** (shared system memory)
* **Distributed processing** (typically no shared memory)

**Import note before we start: At the center of an efficient parallel code is a fast serial code!!**

### Why Go Parallel?

<img src="../../static/42-years-processor-trend.svg" width=700px>

Interesting video on the topic of "The Future of Microprocessors" https://www.youtube.com/watch?v=zX4ZNfvw1cw (coincidentally from Juliacon :P)

### When to Go Parallel?

- If parts of your (optimized!) serial code aren't fast enough.
  - There are costs: parallelization typically increases the code complexity
- If your system has multiple execution units (CPU threads, GPU threads, ...).
  - Import on supercomputers, but also on modern desktop computers and laptops

### What Do I Have?

In [ ]:
using Hwloc
Hwloc.num_physical_cores()

Note that there may be more than one CPU thread per physical CPU core (e.g. hyperthreading).

In [ ]:
Sys.CPU_THREADS

### What does Maxwell Have?

The [Maxwell Infrastructure](https://confluence.desy.de/display/MXW/Infrastructure) page summarises the hardware:

| Compute Hardware                          |               | Infiniband Hardware |        | Storage     |        |
|-------------------------------------------|---------------|---------------------|--------|-------------|--------|
| CPU+GPU nodes                             | 798           | root switches       | 6      | GPFS exfel  | ~40 PB |
| Total number of cores with hyperthreading | 61696         | top switches        | 12     | GPFS petra3 | ~20 PB |
| Total number of PHYSICAL cores            | 30898         | leaf switches       | 42     | BeeGFS desy | 1.5 PB |
| Theoretical CPU peak performance          | 1074 TFlops   | IB cables (#)       | >1432  | BeeGFS cssb | 3.2 PB |
| Total RAM                                 | 420 TB        | IB cables (length)  | >7.6km |             |        |
| GPU nodes                                 | 180           |                     |        |             |        |
| Total number of GPUs                      | 379           |                     |        |             |        |
| Theoretical GPU peak performance          | 2330 TFlops   |                     |        |             |        |
| Total peak performance                    |  3404 TFlops1 |                     |        |             |        |

There are two main kinds of nodes on Maxwell:

| HT Cores | Cores | CPUs | CPU           |
|----------|-------|------|---------------|
| ~160     | ~20   | 2x   | Intel E5-2698 |
|  256     |  64   | 2x   | AMD EPYC 7542 |

Note that:

- Few different types of Intel CPUs, between 18 and 20 cores/cpu
- Hyperthreaded cores = 2 (physical CPUs) * 64 (cores/CPU) * 2 (threads/core) = 256 HT Cores for EPYC, similar for Intel

Even if you only use a single node you have access to 128 CPU cores (64 per CPU). Hence, if you would use only a single core, the node utilization would be less than 1%.

### Amdahl's Law

Naive strong scaling expectation: I have 4 cores, give me my 4x speedup! However that is not the case:

> The overall performance improvement gained by optimizing a single part of a system is limited by the fraction of time that the improved part is actually used

More formally:

>If $p$ is the fraction of a code that can be parallelized than the maximal theoretical speedup by parallelizing on $n$ cores is given by
>$$ F(n) = 1/(1-p + p/n) $$

In [ ]:
using Plots
F(p, n) = 1 / (1 - p + p / n)

pl = plot()
for p in reverse(sort(vcat(0.2:0.2:1, [0.9, 0.95])))
    plot!(pl, n -> F(p, n), 1:16, lab="$(Int(p*100))%", lw=2,
        legend=:topleft, xlab="number of cores", ylab="parallel speedup", frame=:box)
end
pl

## [Parallel Computing](https://docs.julialang.org/en/v1/manual/parallel-computing/) in Julia

Julia provides support for all types of parallelism mentioned above (same order)

* `@simd`, [SIMD.jl](https://github.com/eschnett/SIMD.jl), [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
* `Threads.@threads`, `Threads.@spawn`, [FLoops.jl](https://github.com/JuliaFolds/FLoops.jl), [ThreadsX.jl](https://github.com/tkf/ThreadsX.jl) ...
* `@spawnat`, `@fetch`, `RemoteChannel`, `SharedArray`
* `@spawnat`, `@fetch`, `RemoteChannel`, [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl), [MPI.jl](https://github.com/JuliaParallel/MPI.jl)

With supercomputing in mind, we will start by focusing on multi-process parallelism which allows us to utilize multiple cores on the same or different nodes/machines (distributed computing).

But before we do, it's instructive to take a closer look at **tasks**.

## Tasks

By default, Julia waits for every command to finish ("**blocking**") and run everything sequentially.

**Tasks** are a control flow feature that allows computations to be suspended and resumed in a flexible manner to implement **cooperative multitasking**. (This feature is sometimes called by other names, such as coroutines, green-, or lightweight threads.)

Tasks are managed by Julia and can be run in a **concurrent** fashion.

> **Concurrency** means executing multiple tasks at the same time but not necessarily simultaneously.

An important use case is **asynchronous I/O**, which is typically slow. Examples are:

- **multiple user input** (Why not already process some of the input?)
- **data dumping to disk** (Maybe it's possible to continue a calculation?)
- **receiving calculations from worker processes**

## `@async` and `@sync`

We can create and schedule a task for asynchronous execution with the [`@async` macro](https://docs.julialang.org/en/v1/base/parallel/#Base.@async).

What this means is that for whatever falls into its scope, Julia will start a task to then proceed to whatever comes next in the script without waiting for the task to complete ("**non-blocking**").

In [ ]:
@time sleep(2);

In [ ]:
@time @async sleep(2)

Julia allows the script to proceed (and the `@time` macro to fully execute) without waiting for the task (in this case, sleeping for two seconds) to complete.

We can use the partner macro `@sync` to synchronize, that is wait for all encapsulated tasks. (see `?@sync`). 

In [ ]:
@time @sync @async sleep(2)

Of course, here it doesn't make much sense to write `@sync @async` - we could simply drop it altogether. A better example is the following.

In [ ]:
@time @sync begin
    @async sleep(2.0)
    @async sleep(2.0)
end

In [ ]:
A = rand(1000, 1000)
B = rand(1000, 1000)

@time t = @async A * B;

In [ ]:
@time A * B;

In [ ]:
wait(t)

In [ ]:
fetch(t)

In [ ]:
function io_bound_task()
    sleep(5.0)
    return true
end

In [ ]:
@time my_io_bound_task = @async io_bound_task()

@time fetch(my_io_bound_task)